In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pizza-not-pizza:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2296957%2F3863171%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240425%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240425T034138Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D649aa22de8c7b2cd600ec02b963ee3399bda8889eaba3f4bb0f631c8b7bb4208b395bc20b1c2f164883f447d1cee2644e78596a556fe85ff105dbb153fd6725c0abc5e53ddad4f8e49977296b5f831e7a11a7becea057d369fb7d63f6843220af452b44028624ba70c7a8263c968ae87d45387888712afed6b2a79787cf512470ccef0bce305f0f9e531e76d9431fd10823f44583d242069cd2d386f4d1c8b96dcd58f6f53cb1e59c5c100191f68d413e1a3363d919944ae0ddab4cbcfe1f2a687a417b132db5051ea5af72de675d0e5542fc2e93033ae0a10de40bbd549b02655758bb95ea825abd67fcdb0df58aae0e42762bfd9475973fdd89fac30f28d5e,pizza-detector:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2535894%2F4304910%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240425%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240425T034138Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D38d659fab598e9c62d03ffea25a0d3c1a465eb85e83e51177c06ba7f3799a649bb7a1ec35ceb00318569ee81550bcfe1d4bda68e7eb6aa3a184f5d29129b30013916d7cce4080ae798f47a7bf0d69f25e2855e0357152763c1291f1ba2db8967a8eeb522a73440bdab68dd67479e4f1c4ac480d9a66d40301f68a0225e113671e6853fe6f2db4793955f4c9c44fd6ea931df49e40715e772c201e5a52fca3319b38ffe89b24aa7fe41fd9a055390e7f3afdc811419c3eab5392e966d83b0aa165f073dc685c8e7f81a59b9ea68716a7a56b7deb61d7464ce7e6f1062fc10fc66e073be741a92172601110ea679e6866f57389f58ec92cae30b45ebddcc9c6ebb'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# **Imports**

In [ ]:
# Common
import os
import numpy as np
import pandas as pd
import tensorflow as tf

# Data
from keras.preprocessing.image import ImageDataGenerator

# Data Visualization
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

# Model
from keras.models import Sequential, load_model
from keras.layers import Dense, GlobalAvgPool2D, Dropout

# Callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Pretrained model
from tensorflow.keras.applications import ResNet50V2, InceptionV3, Xception, ResNet152V2

# **Data - Info**

In [ ]:
rootpath = '../input/pizza-not-pizza/pizza_not_pizza/'

# Get Class Names
class_names = os.listdir(rootpath)[:2]
class_names

There are **2 class**, but we are more interested in the **class distribution**.

In [ ]:
# Calculate Class Distributiona
class_dis = [len(os.listdir(rootpath + name)) for name in class_names]

# Plot
fig = px.pie(names=class_names, values=class_dis, title="Class Distribution")
fig.update_layout({'title':{'x':0.5}})
fig.show()

In [ ]:
# Plot
fig = px.bar(x=class_names, y=class_dis, title="Class Distribution")
fig.update_layout({'title':{'x':0.5}})
fig.show()

**Superb!!**. Both the **classes are equally distributed** this ensures that the **model cannot be biased towards any class**.

# **Data Loading**

In [ ]:
# Initialize Generator
gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    validation_split=0.2
)

# Load Data
train_ds = gen.flow_from_directory(
    rootpath,
    target_size=(256,256),
    subset='training',
    batch_size=32,
    shuffle=True,
    class_mode='binary'
)

valid_ds = gen.flow_from_directory(
    rootpath,
    target_size=(256,256),
    subset='validation',
    batch_size=32,
    shuffle=True,
    class_mode='binary'
)

# **Data Visualization**

The best way to understand the **data is to Visualize it**.

In [ ]:
i=1
plt.figure(figsize=(20,10))
for images, labels in train_ds:

    id = np.random.randint(len(images))
    image, label = images[id], int(labels[id])

    plt.subplot(4,10,i)
    plt.imshow(image)
    plt.title(f"Class : {class_names[label]}")
    plt.axis('off')

    i+=1
    if i>=41:
        break
plt.tight_layout()
plt.show()

This will be a **tough task**, this is because the **"not pizza"** contains a **wide variety of food items**.

# **Inception**

In [ ]:
base_model = InceptionV3(include_top=False, input_shape=(256,256,3))

# Freeze Weights
base_model.trainable = False

# Model Architecture
model = Sequential([
    base_model,
    GlobalAvgPool2D(),
    Dense(256, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
], name="Inception-TL")

# Compile Model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Callbacks
cbs = [EarlyStopping(patience=3, restore_best_weights=True), ModelCheckpoint("Inception-TL.h5", save_best_only=True)]

# Model Training
# model.fit(train_ds, validation_data=valid_ds, epochs=50, callbacks=cbs)

In [ ]:
model.evaluate(train_ds)

In [ ]:
model.evaluate(valid_ds)

# **Prediction**

In [ ]:
model = load_model('../input/pizza-detector/Inception-Pizza-Detector.h5')

In [ ]:
model.summary()

In [ ]:
i=1
plt.figure(figsize=(20,20))
for images, labels in valid_ds:

    id = np.random.randint(len(images))
    image, label = images[id], int(labels[id])
    pred_label = int(np.round(model.predict(image.reshape(-1,256,256,3))))

    plt.subplot(4,5,i)
    plt.imshow(image)
    plt.title(f"Class : {class_names[label]}, Pred : {class_names[pred_label]}")
    plt.axis('off')

    i+=1
    if i>=21:
        break
plt.tight_layout()
plt.show()